# Portfolio Statistics Aggregator

This notebook lets you input the summary statistics for each period (e.g. each 7‑day period) and then computes an aggregated overall statistic for the full combined period. The input statistics for each period are assumed to be:

- **Final Portfolio Value** (as a growth factor over the period, e.g. 1.0179 means a 1.79% gain)
- **Annualized Sharpe Ratio** (calculated from the 2‑minute returns within that period using the formula: 
  ```python
  ann_sharpe = (mean_ret / std_ret) * np.sqrt(195 * 252)
  ```
  where 195 is the number of intervals per day and 252 is the number of trading days per year)
- **Maximum Drawdown** (given as a negative percentage, e.g. -7.39%)
- **Win Rate** (the percentage of intervals with positive returns)

### Aggregation Method

- **Final Portfolio Value:** The overall growth factor is computed as the product of each period's final portfolio value.
- **Annualized Sharpe Ratio:** As an approximation, the overall Sharpe is taken as the simple average of the period Sharpe ratios (this is only an estimate since Sharpe ratios are not directly additive).
- **Maximum Drawdown:** We take the worst (most negative) drawdown across the periods as the combined drawdown.
- **Win Rate:** With equal-length periods, the overall win rate is approximated by the average of the period win rates.

Feel free to adjust these methods if you have additional information (such as the number of intervals per period) or if you have the underlying return series available.

In [2]:
import numpy as np
import pandas as pd

# --- Load weekly stats from CSV ---
file_path = "weekly_portfolio_stats.csv"
df = pd.read_csv(file_path)

print("Input Period Statistics:")
print(df)

# --- Aggregating the Statistics ---

# 1. Final Portfolio Value: Multiply the period values to get the overall growth factor
combined_final_value = np.prod(df['Final_Portfolio_Value'])

# 2. Annualized Sharpe Ratio: Average of individual Sharpe ratios
combined_sharpe = df['Annualized_Sharpe'].mean()

# 3. Maximum Drawdown: Worst (most negative)
combined_drawdown = df['Max_Drawdown'].min()

# 4. Win Rate: Average across periods
combined_win_rate = df['Win_Rate'].mean()

# --- Calculate Annualized Return ---
# Each period is 7 days (weekly rebalancing)
total_days = len(df) * 7
annualization_factor = 252 / total_days
annualized_return = combined_final_value ** annualization_factor - 1

# --- Display the Aggregated Results ---
print("\nCombined Portfolio Performance Across Periods:")
print(f"Combined Final Portfolio Value : {combined_final_value:.4f}")
print(f"Combined Annualized Sharpe     : {combined_sharpe:.4f}")
print(f"Combined Maximum Drawdown      : {combined_drawdown:.2%}")
print(f"Combined Win Rate              : {combined_win_rate:.2%}")
print(f"Estimated Annualized Return    : {annualized_return * 100:.2f}%")


Input Period Statistics:
   Final_Portfolio_Value  Annualized_Sharpe  Max_Drawdown  Win_Rate  \
0               1.040303           3.814032     -0.067749  0.530130   
1               1.052549           4.738009     -0.037012  0.516081   
2               1.032088           3.149347     -0.080200  0.514260   
3               1.018754           2.080986     -0.074128  0.497364   

   Period_End                     Method  
0  2025-03-08  max_return_for_volatility  
1  2025-03-15  max_return_for_volatility  
2  2025-03-22  max_return_for_volatility  
3  2025-03-29  max_return_for_volatility  

Combined Portfolio Performance Across Periods:
Combined Final Portfolio Value : 1.1513
Combined Annualized Sharpe     : 3.4456
Combined Maximum Drawdown      : -8.02%
Combined Win Rate              : 51.45%
Estimated Annualized Return    : 255.38%
